# Webscraping data from http://www.aduana.gov.py/datos/

## Main - Webscrapes import/export data from Paraguay's aduana website going around (easy) captcha image.

### Autor: Henrique Guarneri

In [1]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import pytesseract
import pandas as pd

In [2]:
class Session():
    
    def __init__(self):
        pass
    def createSession(self):
        self.session = requests.Session()

class Captcha(object):
    
    def __init__(self):
        pass
            
    def getCaptcha(self,url,session):
        image = session.get(url).content
        img = BytesIO(image)
        im = Image.open(img)
        self.image = im
    
    def readCaptcha(self):
        im = self.image.convert('1')
        self.text = pytesseract.image_to_string(im)

class ParametersAduana():
    
    def __init__(self, mercaderia, tipo_op, periodo, radio, mercaderia2, captcha):
        
        self.mercaderia = mercaderia
        self.tipo_op = tipo_op
        self.periodo = periodo
        self.radio = radio
        self.mercaderia2 = mercaderia2
        self.captcha = captcha
        self.parameters = {'mercaderia':self.mercaderia, 'tipo_op':self.tipo_op ,'periodo':self.periodo, 'radio':self.radio,
                'mercaderia2':self.mercaderia2 ,'captcha':self.captcha}
        
class Table():
    
    def __init__(self):
        pass
    
    def setParameters(self,parameters):
        ### parameters = dict
        
        self.parameters = parameters
    
    def getTable(self,url,session):
        
        self.postResponse = session.post(url, data=self.parameters)
        self.htmlTable = BeautifulSoup(self.postResponse.text, 'html5lib')
        self.DataFrame = pd.read_html(self.postResponse.text,encoding='ISO-8859-1')[0]
    
    def saveTable(self,path):
        
        self.DataFrame.to_csv(path,encoding='ISO-8859-1', sep='\t')
        
def core(url,cUrl,path,mercaderia,tipo_op,periodo,radio,mercaderia2,captcha):
    
    #print('### SETTING PARAMETERS ###')
        
    p = ParametersAduana(mercaderia, tipo_op, periodo, radio, mercaderia2, captcha)
        
    #print('### CREATING SESSION ###')
        
    s = Session()
    s.createSession()
        
    #print('### HACKING CAPTCHA###')
        
    c = Captcha()
    c.getCaptcha(cUrl,s.session)
    c.readCaptcha()
    p.parameters['captcha'] = c.text
        
    #print('### CREATING TABLE ###')
        
    t = Table()
    t.setParameters(p.parameters)
        
    #print('### GETTING TABLE ###')
        
    t.getTable(url,s.session)
        
    #print('### SAVING TABLE ###')
        
    t.saveTable(path + 'table_'+mercaderia+'_'+periodo+'.csv')
        
    #print('### SUCCESS ###')
  
    
   

In [3]:
if __name__=="__main__":
    
    url='http://www.aduana.gov.py/datos/index2020b.php'
    cUrl = 'http://www.aduana.gov.py/datos/captcha.php'
    path = 'tabelas//'
    ncm = pd.read_table('Paraguay NCM.txt',header = None)
    mercaderiaList = ncm[0].tolist()
    tipo_op = ''
    periodo = '2015'
    radio = '99'
    mercaderia2 = ''
    captcha = ''
    
    log = file('log.txt', 'a')
    
    for mercaderia in mercaderiaList:
        captcha = ''
        try:
            core(url,cUrl,path,str(mercaderia),tipo_op,periodo,radio,mercaderia2,captcha)
        except:
            log.write(str(mercaderia)+"\n")
    log.close()
            

C:\Anaconda\lib\site-packages\bs4\__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
